In [644]:
import gower
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from scipy.optimize import linear_sum_assignment

In [645]:
og_df = pd.read_csv("datasets/heart_disease_uci.csv")
og_df.drop(columns=["id", "dataset"], inplace=True)
og_df = og_df.drop(og_df[og_df["num"] == 0].sample(frac=0.3).index)
og_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,46,Male,asymptomatic,134.0,310.0,False,normal,126.0,False,0.0,NaN,NaN,normal,2
915,54,Female,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
917,55,Male,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [646]:
categorial_columns = ["sex", "cp", "fbs", "restecg", "exang", "slope", "thal"]
cont_columns = ["age", "trestbps", "chol", "thalch", "oldpeak", "ca"]

In [647]:
df = og_df.copy()
df.drop(columns="num", inplace=True)
df[categorial_columns] = df[categorial_columns].apply(LabelEncoder().fit_transform)
df[categorial_columns] = MinMaxScaler().fit_transform(df[categorial_columns])

df[cont_columns] = MinMaxScaler().fit_transform(df[cont_columns])
df = df.fillna(df.mean())
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,0.714286,1.0,1.000000,0.725000,0.386401,0.5,0.000000,0.666667,0.0,0.556818,0.000000,0.000000,0.000000
1,0.795918,1.0,0.000000,0.800000,0.474295,0.0,0.000000,0.355556,0.5,0.465909,0.333333,1.000000,0.333333
2,0.795918,1.0,0.000000,0.600000,0.379768,0.0,0.000000,0.511111,0.5,0.590909,0.333333,0.666667,0.666667
3,0.183673,1.0,0.666667,0.650000,0.414594,0.0,0.333333,0.940741,0.0,0.693182,0.000000,0.000000,0.333333
4,0.265306,0.0,0.333333,0.650000,0.338308,0.0,0.000000,0.829630,0.0,0.454545,0.666667,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,0.367347,1.0,0.000000,0.670000,0.514096,0.0,0.333333,0.488889,0.0,0.295455,1.000000,0.244275,0.333333
915,0.530612,0.0,0.000000,0.635000,0.552239,0.5,0.666667,0.696296,0.0,0.295455,1.000000,0.244275,1.000000
917,0.551020,1.0,0.000000,0.610000,0.369818,0.5,0.666667,0.296296,0.0,0.295455,1.000000,0.244275,0.000000
918,0.612245,1.0,0.000000,0.662809,0.638474,0.5,0.000000,0.561071,1.0,0.402291,1.000000,0.244275,1.000000


In [648]:
class HeartDiseaseDataset(Dataset):
    def __init__(self, df):
        self.x = torch.tensor(df.values, dtype=torch.float)

    def __getitem__(self, idx):
        return self.x[idx]

    def __len__(self):
        return self.x.shape[0]

In [649]:
dataset = HeartDiseaseDataset(df)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)
len(dataset)

797

In [650]:
class MixedTypeClusteringModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(13, 10),
            torch.nn.Sigmoid(),
            torch.nn.Linear(10, 7),
            torch.nn.Sigmoid(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(7, 10),
            torch.nn.Sigmoid(),
            torch.nn.Linear(10, 13),
            torch.nn.Sigmoid()
        )


    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


epochs = 20
lr = 0.001

model = MixedTypeClusteringModel()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    loss = 0

    for x in dataloader:
        optimizer.zero_grad()
        outputs = model(x)
        train_loss = criterion(outputs,  x)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()

    loss = loss / len(dataloader)
    print("epoch: {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch: 1/20, loss = 0.130076
epoch: 2/20, loss = 0.124864
epoch: 3/20, loss = 0.120041
epoch: 4/20, loss = 0.115587
epoch: 5/20, loss = 0.111498
epoch: 6/20, loss = 0.107711
epoch: 7/20, loss = 0.104189
epoch: 8/20, loss = 0.100870
epoch: 9/20, loss = 0.097821
epoch: 10/20, loss = 0.094940
epoch: 11/20, loss = 0.092265
epoch: 12/20, loss = 0.089810
epoch: 13/20, loss = 0.087511
epoch: 14/20, loss = 0.085409
epoch: 15/20, loss = 0.083459
epoch: 16/20, loss = 0.081664
epoch: 17/20, loss = 0.080058
epoch: 18/20, loss = 0.078594
epoch: 19/20, loss = 0.077258
epoch: 20/20, loss = 0.076068


In [651]:
features = model.encoder(torch.tensor(df.values, dtype=torch.float)).detach().numpy()
features

array([[0.6276887 , 0.3532071 , 0.43400538, ..., 0.8793764 , 0.84032094,
        0.22467731],
       [0.6201081 , 0.35701543, 0.43718317, ..., 0.8727474 , 0.83582693,
        0.23269871],
       [0.6209049 , 0.3553431 , 0.43865177, ..., 0.87172484, 0.83343977,
        0.23284464],
       ...,
       [0.6260015 , 0.35257292, 0.43360898, ..., 0.87858534, 0.8406184 ,
        0.22630098],
       [0.6292244 , 0.3451586 , 0.43417776, ..., 0.8842247 , 0.84315264,
        0.21648553],
       [0.62792003, 0.3507061 , 0.43594486, ..., 0.8761832 , 0.83435357,
        0.22694758]], dtype=float32)

In [652]:
def cluster_accuracy(y_pred, y_true):
    # We need to map the labels to our cluster labels
    # This is a linear assignment problem on a bipartite graph
    k = max(len(np.unique(y_pred)), len(np.unique(y_pred)))
    cost_matrix = np.zeros((k, k))
    for i in range(y_pred.size):
        cost_matrix[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(cost_matrix.max() - cost_matrix)
    return cost_matrix[row_ind, col_ind].sum() / y_pred.size

In [653]:
autoencoder_kmeans = KMeans(n_clusters=5, n_init="auto", random_state=0).fit(features)
autoencoder_acc = cluster_accuracy(autoencoder_kmeans.labels_, og_df["num"].to_numpy())
autoencoder_acc

0.28983688833124216

In [654]:
def encode_feature(df, feature_to_encode):
    dummies = pd.get_dummies(df[[feature_to_encode]])
    result_df = pd.concat([df, dummies], axis=1)
    result_df.drop(columns=feature_to_encode, inplace=True)
    return result_df

In [655]:
df_one_hot = og_df.copy()
df_one_hot.drop(columns="num", inplace=True)

for col in categorial_columns:
    df_one_hot = encode_feature(df_one_hot, col)
df_one_hot.loc[:, ~df_one_hot.columns.isin(cont_columns)] = df_one_hot.loc[:, ~df_one_hot.columns.isin(cont_columns)].apply(LabelEncoder().fit_transform)
df_one_hot = df_one_hot.fillna(df_one_hot.mean())
df_one_hot

,age,trestbps,chol,thalch,oldpeak,ca,sex_Female,sex_Male,cp_asymptomatic,cp_atypical angina,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,63,145.000000,233.0,150.000000,2.300000,0.000000,0,1,0,0,...,0,0,1,0,1,0,0,1,0,0
1,67,160.000000,286.0,108.000000,1.500000,3.000000,0,1,1,0,...,0,0,0,1,0,1,0,0,1,0
2,67,120.000000,229.0,129.000000,2.600000,2.000000,0,1,1,0,...,0,0,0,1,0,1,0,0,0,1
3,37,130.000000,250.0,187.000000,3.500000,0.000000,0,1,0,0,...,1,0,1,0,1,0,0,0,1,0
4,41,130.000000,204.0,172.000000,1.400000,0.000000,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,46,134.000000,310.0,126.000000,0.000000,0.732824,0,1,1,0,...,1,0,1,0,0,0,0,0,1,0
915,54,127.000000,333.0,154.000000,0.000000,0.732824,1,0,1,0,...,0,1,1,0,0,0,0,0,0,0
917,55,122.000000,223.0,100.000000,0.000000,0.732824,0,1,1,0,...,0,1,1,0,0,0,0,1,0,0
918,58,132.561828,385.0,135.744652,0.940162,0.732824,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [656]:
classic_one_hot_kmeans = KMeans(n_clusters=5, n_init="auto", random_state=0).fit(df_one_hot)
classic_one_hot_acc = cluster_accuracy(classic_one_hot_kmeans.labels_, og_df["num"].to_numpy())
classic_one_hot_acc

0.3161856963613551

In [657]:
one_hot_dataset = HeartDiseaseDataset(df_one_hot)
one_hot_dataloader = DataLoader(one_hot_dataset, batch_size=100, shuffle=True)
len(one_hot_dataset)

797

In [658]:
class OneHotMixedTypeClusteringModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(25, 18),
            torch.nn.Sigmoid(),
            torch.nn.Linear(18, 12),
            torch.nn.Sigmoid(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(12, 18),
            torch.nn.Sigmoid(),
            torch.nn.Linear(18, 25),
            torch.nn.Sigmoid()
        )


    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


epochs = 20
lr = 0.001

one_hot_model = OneHotMixedTypeClusteringModel()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    loss = 0

    for x in one_hot_dataloader:
        optimizer.zero_grad()
        outputs = one_hot_model(x)
        train_loss = criterion(outputs,  x)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()

    loss = loss / len(dataloader)
    print("epoch: {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch: 1/20, loss = 3621.528656
epoch: 2/20, loss = 3620.895966
epoch: 3/20, loss = 3619.893982
epoch: 4/20, loss = 3621.468842
epoch: 5/20, loss = 3621.529175
epoch: 6/20, loss = 3620.428436
epoch: 7/20, loss = 3621.433075
epoch: 8/20, loss = 3620.236633
epoch: 9/20, loss = 3620.360809
epoch: 10/20, loss = 3621.693237
epoch: 11/20, loss = 3620.456696
epoch: 12/20, loss = 3620.957001
epoch: 13/20, loss = 3620.958038
epoch: 14/20, loss = 3620.763611
epoch: 15/20, loss = 3620.787537
epoch: 16/20, loss = 3620.688843
epoch: 17/20, loss = 3621.290863
epoch: 18/20, loss = 3621.613983
epoch: 19/20, loss = 3621.647095
epoch: 20/20, loss = 3621.391632


In [659]:
one_hot_features = one_hot_model.encoder(torch.tensor(df_one_hot.values, dtype=torch.float)).detach().numpy()
one_hot_autoencoder_kmeans = KMeans(n_clusters=5, n_init="auto", random_state=0).fit(one_hot_features)
one_hot_autoencoder_acc = cluster_accuracy(one_hot_autoencoder_kmeans.labels_, og_df["num"].to_numpy())
one_hot_autoencoder_acc

0.38143036386449186

In [660]:
no_nan_df = og_df.interpolate()
distance_matrix = gower.gower_matrix(no_nan_df)
gower_agglo = AgglomerativeClustering(n_clusters=5, metric="precomputed", linkage="average").fit_predict(distance_matrix)
gower_agglo

array([0, 0, 0, 4, 4, 0, 4, 0, 0, 0, 4, 0, 4, 0, 4, 4, 4, 4, 4, 0, 4, 4,
       0, 4, 0, 4, 0, 4, 4, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4,
       0, 0, 0, 0, 4, 4, 0, 4, 0, 4, 0, 0, 4, 4, 0, 0, 4, 0, 0, 0, 4, 4,
       0, 4, 0, 4, 0, 4, 4, 4, 4, 0, 4, 0, 0, 0, 4, 4, 4, 4, 0, 0, 4, 0,
       0, 0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, 0, 4, 0, 0, 4, 4, 4, 0, 0, 4,
       0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 4, 4, 0, 4, 4, 0, 4, 0, 0, 0, 4, 0, 0, 4, 0, 4, 4, 0,
       0, 4, 4, 4, 0, 0, 4, 0, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0,
       4, 0, 4, 4, 0, 4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 0, 0, 0, 4, 4, 0, 0,
       0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 4, 4, 0, 4,
       4, 4, 0, 0, 0, 4, 0, 4, 0, 0, 4, 4, 4, 4, 4, 4, 0, 4, 0, 4, 0, 3,
       0, 4, 4, 4, 0, 0, 0, 4, 0, 0, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 0,

In [661]:
gower_agglo_acc = cluster_accuracy(gower_agglo, og_df["num"].to_numpy())
gower_agglo_acc

0.5395232120451694

In [662]:
print(f"Naively choose most frequent label: {og_df['num'].value_counts().max()/og_df['num'].count()}")
print(f"Classic Kmeans with OneHot: {classic_one_hot_acc}")
print(f"Gower Distance Agglomerative: {gower_agglo_acc}")
print(f"Autoencoder Kmeans: {autoencoder_acc}")
print(f"Autoencoder Kmeans and OneHot: {one_hot_autoencoder_acc}")

Naively choose most frequent label: 0.3613550815558344
Classic Kmeans with OneHot: 0.3161856963613551
Gower Distance Agglomerative: 0.5395232120451694
Autoencoder Kmeans: 0.28983688833124216
Autoencoder Kmeans and OneHot: 0.38143036386449186
